In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-06-01'
final_date = '2022-06-30'
query_error_year = 2022
query_error_month = 6

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\06. JUNIO\DATA\ERROR JUNIO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 4)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(87050, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10896\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
79129,79129,a62b2198-f74a-46b0-9bdf-ad2f66e3dc5e,Pozas Lau,LW9DSHG87,2022-06-01,0.02,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,a62b2198-f74a-46b0-9bdf-ad2f66e3dc5e,2022-06-01,1,Actual
79136,79136,3ca0ae40-e5fe-4763-a316-3a28e8f66cd1,abarrotes la avenida,Q7MBBGVDZ,2022-06-01,1.80,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,3ca0ae40-e5fe-4763-a316-3a28e8f66cd1,2022-06-01,1,Actual
79135,79135,58019765-6dd5-447b-b080-dd7881a7c0b2,Lety Diamante,VB68P8Q23,2022-06-01,4.80,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,58019765-6dd5-447b-b080-dd7881a7c0b2,2022-06-01,1,Actual
79134,79134,b1a2d0a0-d1f0-4cca-a196-20bacb5927f5,El Patron,62C6N5XS7,2022-06-01,1.39,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,b1a2d0a0-d1f0-4cca-a196-20bacb5927f5,2022-06-01,1,Actual
79133,79133,93eae658-ea8b-4a9f-95a8-6a82f1dd1c88,Tintosol,7KZGLPSF7,2022-06-01,69.31,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,93eae658-ea8b-4a9f-95a8-6a82f1dd1c88,2022-06-01,1,Actual


In [9]:
query_error_current_month.shape

(8151, 17)

In [10]:
query_error_current_month['month'].unique()

array([6], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10896\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
58801,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN
59379,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN
60428,60428,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,74PPCDB2Z,2022-02-28,565.56,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN
60951,60951,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,7VV87XQ2P,2022-03-04,79.53,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN
61764,61764,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,RMRQFHDYW,2022-03-08,178.22,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN


In [15]:
query_new_last_month.shape

(9, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\06. JUNIO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\05. MAY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\05. MAYO\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

## ALL FILES

In [26]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
29552,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,2022-04-30 08:07:58,BANORTE,"Ana Napoles Rodr,,130.32,127180013581467334,CL...",BNTE 2022-04-30.csv,2022-04-30,TRAD
77985,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,2022-04-30 08:14:07,BANORTE,"Luis Carlos Davila Cruz,,210.81,01206002851713...",BNTE 2022-04-30.csv,2022-04-30,TRAD
88961,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,2022-04-30 08:15:28,BANORTE,"Francisco tafoya garcia,,383.30,00218070150924...",BNTE 2022-04-30.csv,2022-04-30,TRAD
2004,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,2022-04-30 08:03:45,BANORTE,"Ivan Fernando Robles Valdez,,862.42,0127600140...",BNTE 2022-04-30.csv,2022-04-30,TRAD
50324,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,2022-04-30 08:10:35,BANORTE,"Juan Ibanez Cortes,,680.34,002180447000535878,...",BNTE 2022-04-30.csv,2022-04-30,TRAD


In [27]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [28]:
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
29552,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,2022-04-30 08:07:58,BANORTE,"Ana Napoles Rodr,,130.32,127180013581467334,CL...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
77985,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,2022-04-30 08:14:07,BANORTE,"Luis Carlos Davila Cruz,,210.81,01206002851713...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
88961,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,2022-04-30 08:15:28,BANORTE,"Francisco tafoya garcia,,383.30,00218070150924...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
2004,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,2022-04-30 08:03:45,BANORTE,"Ivan Fernando Robles Valdez,,862.42,0127600140...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
50324,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,2022-04-30 08:10:35,BANORTE,"Juan Ibanez Cortes,,680.34,002180447000535878,...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día


In [29]:
allfiles.shape

(5386917, 17)

# CRUCE 1 - MES ACTUAL

In [30]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [31]:
cruce_1.shape

(5386917, 25)

In [32]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10896\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2201601,077d3cb3-79c3-4630-bd5f-9904f5be7f70,Imperio Boutique,erpromotionsgdl@gmail.com,ZH942GY5L,2022-05-24,"6,195.02","Banco Santander, S.A.",2022-05-24 10:05:02,Rosa Isela Torres Paredes,'014416605875112742,...,TRAD,Pago del día,"6,195.02",2022-06-03,2022-06-03,6.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2355157,077d3cb3-79c3-4630-bd5f-9904f5be7f70,Imperio Boutique,erpromotionsgdl@gmail.com,SPQRLY4NW,2022-05-26,"4,392.58","Banco Santander, S.A.",2022-05-26 10:36:43,Rosa Isela Torres Paredes,'014416605875112742,...,TRAD,Pago del día,"4,392.58",2022-06-03,2022-06-03,6.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2400756,a4591f18-b475-4e63-bdaa-534657b12571,Miscelanea Samarath,salvadorlsw@hotmail.com,BDMLKDVFV,2022-05-27,562.51,"Opciones Empresariales del Noreste, S.A. de C....",2022-05-27 10:32:24,Giovanni Salvador Luna Yepez,'659802020000110396,...,FP,Pago del día,562.51,2022-06-02,2022-06-02,6.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2401985,a4591f18-b475-4e63-bdaa-534657b12571,Miscelanea Samarath,salvadorlsw@hotmail.com,BRRZ42PRK,2022-05-27,69.00,"Opciones Empresariales del Noreste, S.A. de C....",2022-05-27 14:16:58,Giovanni Salvador Luna Yepez,'659802020000110396,...,FP,Pago del día,69.00,2022-06-02,2022-06-02,6.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2451709,a4591f18-b475-4e63-bdaa-534657b12571,Miscelanea Samarath,salvadorlsw@hotmail.com,VKSYBKF7X,2022-05-27,73.79,"Opciones Empresariales del Noreste, S.A. de C....",2022-05-27 16:27:53,Giovanni Salvador Luna Yepez,'659802020000110396,...,FP,Pago del día,73.79,2022-06-02,2022-06-02,6.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW


In [33]:
cruce_1_final.shape

(2434, 25)

In [34]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [35]:
duplicates_cruce_1_final.shape

(0, 25)

# CRUCE 2 MES ACTUAL

In [36]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,79129,a62b2198-f74a-46b0-9bdf-ad2f66e3dc5e,Pozas Lau,LW9DSHG87,2022-06-01,0.02,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,a62b2198-f74a-46b0-9bdf-ad2f66e3dc5e,2022-06-01,1,Actual,Not found,NaN
1,79136,3ca0ae40-e5fe-4763-a316-3a28e8f66cd1,abarrotes la avenida,Q7MBBGVDZ,2022-06-01,1.80,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,3ca0ae40-e5fe-4763-a316-3a28e8f66cd1,2022-06-01,1,Actual,Not found,NaN
2,79135,58019765-6dd5-447b-b080-dd7881a7c0b2,Lety Diamante,VB68P8Q23,2022-06-01,4.80,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,58019765-6dd5-447b-b080-dd7881a7c0b2,2022-06-01,1,Actual,Not found,NaN
3,79134,b1a2d0a0-d1f0-4cca-a196-20bacb5927f5,El Patron,62C6N5XS7,2022-06-01,1.39,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,b1a2d0a0-d1f0-4cca-a196-20bacb5927f5,2022-06-01,1,Actual,Not found,NaN
4,79133,93eae658-ea8b-4a9f-95a8-6a82f1dd1c88,Tintosol,7KZGLPSF7,2022-06-01,69.31,ERROR,Bank account information not available,2022-06-01,6,2022,NEW,SIN_CLABE,93eae658-ea8b-4a9f-95a8-6a82f1dd1c88,2022-06-01,1,Actual,Not found,NaN


In [37]:
cruce_2.shape

(8417, 19)

In [38]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
12,79125,d8e1fa40-916e-4896-b62a-749c3f68c58f,Abarrotes Torrecitas,H8FDPN345,2022-06-01,877.74,ERROR,DA,2022-06-01,6,2022,NEW,ERROR_USUAL,d8e1fa40-916e-4896-b62a-749c3f68c58f,2022-06-01,1,Actual,TRAD,Pago del día
13,79124,b041eaad-1164-4ba0-abcf-17a943c586fc,Mini Super Sofi Villa,3K7Z6C3HZ,2022-06-01,173.45,ERROR,DB,2022-06-01,6,2022,NEW,ERROR_USUAL,b041eaad-1164-4ba0-abcf-17a943c586fc,2022-06-01,1,Actual,TRAD,Pago del día
15,79122,9403883b-78e0-4f37-b94c-a89697c21560,Abrt Sul y Dan,W5DF27GZ9,2022-06-01,28.75,ERROR,DB,2022-06-01,6,2022,NEW,ERROR_USUAL,9403883b-78e0-4f37-b94c-a89697c21560,2022-06-01,1,Actual,TRAD,Pago del día
16,79121,6fda2b9a-9a3e-41bd-9aae-d73d95b3e4a0,Abarrotes Y Carnes Luevano,6X2DY8M89,2022-06-01,183.21,ERROR,DB,2022-06-01,6,2022,NEW,ERROR_USUAL,6fda2b9a-9a3e-41bd-9aae-d73d95b3e4a0,2022-06-01,1,Actual,TRAD,Pago del día
19,79138,7a60fbcf-388f-47d6-9c29-b99105ea2b13,Torre Oceana,R9FNG49FN,2022-06-01,124.15,ERROR,IF,2022-06-01,6,2022,NEW,ERROR_USUAL,7a60fbcf-388f-47d6-9c29-b99105ea2b13,2022-06-01,1,Actual,TRAD,Pago del día


In [39]:
cruce_2_final.shape

(2700, 19)

In [40]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
205,60876,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,ERROR,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,Actual,TRAD,Reprogramación
206,60876,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,ERROR,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,Actual,TRAD,Reprogramación
207,63612,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,ERROR,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,Actual,TRAD,Reprogramación
208,63612,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,ERROR,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,Actual,TRAD,Reprogramación
209,68809,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,YS947SHFX,2022-04-18,"6,736.87",ERROR,DM,2022-06-01,6,2022,ERROR,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,Actual,TRAD,Reprogramación


In [41]:
duplicates_cruce_2_final.shape

(454, 19)

In [42]:
duplicates_cruce_2_final.to_csv('duplicates.csv')

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [43]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-06-01           0.00      16,181.77   
1                         2022-06-02       5,244.47       5,527.14   
2                         2022-06-03           0.00           0.00   
3                         2022-06-04           0.00           0.00   
4                         2022-06-05           0.00           0.00   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                       0.00           0.00    94,900.90           0.00   
1                   1,242.88           0.00    33,428.84           0.00   
2                       0.00           0.00    49,934.98           0.00   
3                       0.00           0.00     4,393.61           0.00   
4                       0.00           0.00     3,465.58           0.00   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0               111,082.67  
1                45,443.33  
2                49,934.98  
3                 4,393.61  
4                 3,465.58

In [44]:
group_tipo_pago.shape

(31, 8)

In [45]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-06-01      16,181.77    94,900.90           0.00   
1                       2022-06-02      10,771.61    34,671.72           0.00   
2                       2022-06-03           0.00    49,934.98           0.00   
3                       2022-06-04           0.00     4,393.61           0.00   
4                       2022-06-05           0.00     3,465.58           0.00   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             111,082.67  
1              45,443.33  
2              49,934.98  
3               4,393.61  
4               3,465.58

In [46]:
group_esquema.shape

(31, 5)

# CRUCE 3 - MES ANTERIOR

In [47]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [48]:
cruce_3.shape

(5386917, 25)

In [49]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10896\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
372922,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,itzmari20@gmail.com,74PPCDB2Z,2022-02-28,565.56,"Banco Compartamos, S.A.",2022-05-04 08:15:37,IRIS ITZEL LOPEZ MARI,'130610001489673485,...,TRAD,Reprogramación,565.56,2022-05-04,2022-05-04,5.00,ERROR,DI,ERROR_USUAL,NEW
375256,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,itzmari20@gmail.com,7VV87XQ2P,2022-03-04,79.53,"Banco Compartamos, S.A.",2022-05-04 08:15:37,IRIS ITZEL LOPEZ MARI,'130610001489673485,...,TRAD,Reprogramación,79.53,2022-05-04,2022-05-04,5.00,ERROR,DI,ERROR_USUAL,NEW
382265,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,itzmari20@gmail.com,B6KV2GQRC,2022-02-23,197.39,"Banco Compartamos, S.A.",2022-05-04 08:15:37,IRIS ITZEL LOPEZ MARI,'130610001489673485,...,TRAD,Reprogramación,197.39,2022-05-04,2022-05-04,5.00,ERROR,DI,ERROR_USUAL,NEW
400930,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,itzmari20@gmail.com,JBPV2ZZLD,2022-02-19,45.99,"Banco Compartamos, S.A.",2022-05-04 08:15:37,IRIS ITZEL LOPEZ MARI,'130610001489673485,...,TRAD,Reprogramación,45.99,2022-05-04,2022-05-04,5.00,ERROR,DI,ERROR_USUAL,NEW
409494,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,itzmari20@gmail.com,M582FR5ZR,2022-03-11,40.25,"Banco Compartamos, S.A.",2022-05-04 08:15:37,IRIS ITZEL LOPEZ MARI,'130610001489673485,...,TRAD,Reprogramación,40.25,2022-05-04,2022-05-04,5.00,ERROR,DI,ERROR_USUAL,NEW


In [50]:
cruce_3_final.shape

(9, 25)

In [51]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [52]:
duplicates_cruce_3_final.shape

(0, 25)

# CRUCE 4 - MES ANTERIOR

In [53]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
1,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
2,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
3,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
4,60428,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,74PPCDB2Z,2022-02-28,565.56,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación


In [54]:
cruce_4.shape

(18, 19)

In [55]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
1,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
2,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
3,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
4,60428,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,74PPCDB2Z,2022-02-28,565.56,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación


In [56]:
cruce_4_final.shape

(18, 19)

In [57]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
1,58801,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,JBPV2ZZLD,2022-02-19,45.99,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
2,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
3,59379,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,B6KV2GQRC,2022-02-23,197.39,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación
4,60428,154fb658-78c9-4c4c-b994-684868eeebb3,IRIS ITZEL LOPEZ MARIN,74PPCDB2Z,2022-02-28,565.56,ERROR,DI,2022-05-04,5,2022,NEW,ERROR_USUAL,154fb658-78c9-4c4c-b994-684868eeebb3,2022-05-04,4,NaN,TRAD,Reprogramación


In [58]:
duplicates_cruce_4_final.shape

(18, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [59]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x         
esquema_query                           ERROR_USUAL      All
status_cubeta                                   NEW         
tipo_dispersion                                TRAD         
tipo_pago                            Reprogramación         
0                         2022-05-04       1,302.04 1,302.04
1                                All       1,302.04 1,302.04

In [60]:
group_tipo_pago_last_month.shape

(2, 3)

In [61]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount         
esquema_query                         ERROR_USUAL      All
status_cubeta                                 NEW         
tipo_pago                          Reprogramación         
0                       2022-05-04       2,604.08 2,604.08
1                              All       2,604.08 2,604.08

In [62]:
group_esquema_last_month.shape

(2, 3)

In [63]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL JUNIO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()